In [ ]:
import numpy as np
import pandas as pd
import xlrd

import sys
import os
folder_name = '历史记录'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import shutil
      
def copyDirectory(src, dest):
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)
        
# copy historical summary data
month = int(sys.argv[1])
#month = 2
cwd = os.getcwd()
print(cwd)

folder_list = ['成品销售明细表','成品仓库进出明细']
term_list = ['品牌']

if (month == 1):
    for term in term_list:
        for folder_name in folder_list:
            src = cwd + '/'+term+'/'+folder_name
            dest = cwd + '/历史记录/'+term+'/'+folder_name
            copyDirectory(src, dest)
else:
    for term in term_list:
        for folder_name in folder_list:
            src= cwd[:-1] + str(month-1) + '/'+term+'/'+folder_name
            dest = cwd + '/历史记录/'+term+'/'+folder_name
            copyDirectory(src, dest)

In [ ]:
# add the historical cumulative selling information
# example

history_folder_flag = False
if os.path.exists('历史记录'):
    history_folder_flag = True

cwd= os.getcwd()
if history_folder_flag:
    # get last month information
    key = '销售收入'
    folder_name = '成品销售明细表'
    input_filename = '历史记录/品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_last_month = pd.read_excel(input_filename, sheet_name = 0)
    
    # rebuild dataframe
    term_names = list(df_last_month.iloc[0,:].dropna())
    #category_names = list(df_last_month.iloc[1,:].dropna().unique())
    #print(category_names)
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_last_month= df_last_month.drop([0,1,2])
    df_last_month.set_index(df_last_month.columns.values[0], inplace=True)
    df_last_month.columns = new_columns
    
    
    # current month information
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    

In [ ]:
#df_last_month

In [ ]:
#expenses_last_month = df_last_month.loc[:,(slice(None),'金额')]
#expenses_current_month = df_current_month.loc[:,(slice(None),'金额')]

expenses_last_month = df_last_month.xs('合计金额', level='category', axis=1)
expenses_current_month = df_current_month.xs('合计金额', level='category', axis=1)

print(expenses_last_month.shape)
print(expenses_current_month.shape)

In [ ]:
#expenses_current_month

In [ ]:
#expenses_last_month

In [ ]:
expenses_cumulate = (pd.concat([expenses_last_month, expenses_current_month], sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

#expenses_cumulate

In [ ]:
columns_names = list(expenses_cumulate)
new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
expenses_cumulate.columns = new_columns
#expenses_cumulate

In [ ]:
df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额', '合计金额':'2合计金额'}, inplace=True)

new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
new_df.sort_index(axis=1, inplace=True)
new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                  '3累计合计金额': '累计合计金额'}, inplace=True)
new_df = new_df.sort_values(by=('所有品牌','累计合计金额'),ascending = False)
#new_df

In [ ]:
N_col= new_df.columns.levels[0].size
new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
              
if '无' in new_columns:
    new_columns = new_columns.drop('无').insert(N_col,'无')
if '未注明' in new_columns:
    new_columns = new_columns.drop('未注明').insert(N_col,'未注明')
    
new_df = new_df.reindex(new_columns, level='term', axis=1)

N_row = new_df.shape[0]
new_rows = new_df.index.drop('总计').insert(N_row, '总计')
new_df = new_df.reindex(new_rows)

new_df = new_df.round(2)
#new_df

In [ ]:
def create_cumulative_info_factory(month, key):
    # read last month info
    folder_name = '成品仓库进出明细'
    input_filename = '历史记录/品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_last_month = pd.read_excel(input_filename, sheet_name = 0)
    
    # rebuild dataframe
    term_names = list(df_last_month.iloc[0,:].dropna())
    category_names = ['数量(片)','金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_last_month= df_last_month.drop([0,1,2])
    df_last_month.set_index(df_last_month.columns.values[0], inplace=True)
    df_last_month.columns = new_columns
    
    
    # read current month information
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['数量(片)','金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    if ((month-1)!=1):
        expenses_last_month = df_last_month.xs('累计金额', level='category', axis=1)
    else:
        expenses_last_month = df_last_month.xs('金额', level='category', axis=1)

    expenses_current_month = df_current_month.xs('金额', level='category', axis=1)

    expenses_cumulate = (pd.concat([expenses_last_month, expenses_current_month], sort=False)
                         .groupby(level=0)
                         .sum()
                         .sort_values(by='所有品牌', ascending =False)
                        )
    
    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['2累计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'数量(片)':'0数量(片)', '金额': '1金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0数量(片)':'数量(片)', '1金额': '金额',
                                      '2累计金额':'累计金额'}, inplace=True)
    
    new_df = new_df.sort_values(by=('所有品牌','累计金额'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



    

In [ ]:
def create_cumulative_info_selling(month, key):
    folder_name = '成品销售明细表'
    input_filename = '历史记录/品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_last_month = pd.read_excel(input_filename, sheet_name = 0)
    
    # rebuild dataframe
    term_names = list(df_last_month.iloc[0,:].dropna())
    #category_names = list(df_last_month.iloc[1,:].dropna().unique())
    #print(category_names)
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_last_month= df_last_month.drop([0,1,2])
    df_last_month.set_index(df_last_month.columns.values[0], inplace=True)
    df_last_month.columns = new_columns
    
    
    # current month information
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    if ((month-1)!=1):
        expenses_last_month = df_last_month.xs('累计合计金额', level='category', axis=1)
    else:
        expenses_last_month = df_last_month.xs('合计金额', level='category', axis=1)
    expenses_current_month = df_current_month.xs('合计金额', level='category', axis=1)

    expenses_cumulate = (pd.concat([expenses_last_month, expenses_current_month], sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额',
                                                '合计金额':'2合计金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                      '3累计合计金额': '累计合计金额'}, inplace=True)
    new_df = new_df.sort_values(by=('所有品牌','累计合计金额'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')

    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    if '未注明' in new_columns:
        new_columns = new_columns.drop('未注明').insert(N_col,'未注明')

    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



In [ ]:
def handle_worksheet_format(writer, df_temp, title):
    workbook = writer.book
    worksheet = writer.sheets['sheet1']    
    
    for idx, col in enumerate(df_temp):  # loop through all columns
        series = df_temp[col]
        max_len = max(( series.astype(str).map(len).max() ,  # len of largest item
                        max( [len(str(x)) for x in series.name])*2  # len of column name/header
                      )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = df_temp.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name))  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)
    
    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})
    worksheet_cols = df_temp.shape[1]+1
    
    if (worksheet_cols <=25):
        range_str = 'A1:'+chr(ord('A')+worksheet_cols-1)+'1'
    else:
        last_col_str = chr(ord('A')+ worksheet_cols // 26 - 1) + chr(ord('A')+ (worksheet_cols-1) % 26) + '1'
        range_str = 'A1:'+last_col_str    
    worksheet.merge_range(range_str,title,header_format) 


In [ ]:
def output_cumulative_info_dataframe(month, key, folder_name, out_path, title=None):
    if not os.path.exists(out_path + '/' + folder_name):
        os.makedirs(out_path + '/'+ folder_name)
        
    output_filename = out_path +'/'+ folder_name+'/统计_'+key+'_品牌分类.xlsx'      
    if title is None:
        month_str = ' ('+ str(month)+'月)'
        title=key + month_str
        
    if (folder_name == '成品仓库进出明细'):
        grouped =  create_cumulative_info_factory(month, key)
    elif (folder_name == '成品销售明细表'):
        grouped = create_cumulative_info_selling(month, key)
    else:
        print('Error for folder name')
        exit()
        
        
    # write to Excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workbook
    grouped.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook    
    
    
    # handle the spreadsheet format
    handle_worksheet_format(writer, grouped, title)       

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
        

In [ ]:
out_path = '累计金额信息'
folder_name = '成品仓库进出明细'

key_list = ['销售成本','免费样板','免费广告']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)


In [ ]:
out_path = '累计金额信息'
folder_name = '成品销售明细表'

key_list = ['销售收入', '挂账样板', '免费样板','免费广告','免费费用','破损赔偿','物资自用']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)

